In [1]:
import torch
import torchvision
import os
import torch.nn as nn
from ddpm.diffusion_model import DiffusionModel

import ddpm.config as _config
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

In [2]:
_config.DEBUG = False

In [3]:
cpt_path = 'epoch_24.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
cpt = torch.load(cpt_path, map_location=device, weights_only=False)
model = DiffusionModel(cpt['config'])
model.load_state_dict(cpt['model'])
model = nn.DataParallel(model).to(device)

model.eval()
print("model loaded")

In [5]:
image_num = 5

In [ ]:
for label in range(10):
    labels = torch.ones(image_num, dtype=torch.long, device=device) * label
    samples = model.module.sample(shape=(image_num, 3, 32, 32), device=device, y=labels)
    # Process all samples
    processed_samples = [((sample + 1) / 2).clip(0, 1) for sample in samples]

    # Create a grid with 5 images per row
    grid = make_grid(processed_samples, nrow=5)

    # Display the grid
    plt.title(f"Label: {label}")
    plt.figure(figsize=(15, 3 * (len(processed_samples) // 5 + 1)))
    plt.imshow(grid.permute(1, 2, 0).cpu())
    plt.axis('off')
    plt.show()
    